This notebook was executed in Google Colab using a A100-GPU

### Start of execution

In [1]:
import time

In [2]:
start = time.time()

# 1. Setting the environment

In [3]:
!pip install -q datasets==2.20.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12

In [4]:
!pip install -q bitsandbytes==0.43.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 18.0 MB/s eta 0:00:00


In [5]:
!pip install -q peft==0.11.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.0 MB/s eta 0:00:00


In [6]:
!pip install -q trl==0.9.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 10.2 MB/s eta 0:00:00


# 2. Import Libraries

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import os
import time
import scipy
import torch
import numpy as np
import transformers
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from trl import SFTTrainer
import bitsandbytes as bnb
from datasets import Dataset
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import image as mpimg
from peft import LoraConfig, PeftConfig
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split

from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)

from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 3. Preparation

In [10]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from google.colab import userdata

In [13]:
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [14]:
path_general = 'drive/MyDrive/Profesional_Academico/Github_Personal/ML_AI_Contents/09.Deep_Learning/46.Finetunning_LLM_Hugging_Face'

# 4. Functions

In [15]:
def generate_prompt(text, sentiment):
    return f"""
            [INST]Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative" [/INST]

            [{text}] = {sentiment}
            """.strip()

In [16]:
def generate_test_prompt(text):
    return f"""
            [INST]Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative" [/INST]

            [{text}] = """.strip()

In [17]:
def predict_sentiment(prompt, pipe):

  result = pipe(prompt, pad_token_id = tokenizer.eos_token_id)

  output = ""

  for seq in result:

    output = output + seq['generated_text']

  output_split = output.split(' = ')

  if len(output_split) == 1:

    return 'No sentiment'

  else:

    sentiment = output_split[1]

    return sentiment

In [18]:
def eval_performance(y, y_pred):

  y_pos = (y == 'positive')*1
  y_pos_pred = (y_pred == 'positive')*1

  y_neu = (y == 'neutral')*1
  y_neu_pred = (y_pred == 'neutral')*1

  y_neg = (y == 'negative')*1
  y_neg_pred = (y_pred == 'negative')*1

  print('POSITIVE')
  print('\n')
  print('Recall:', round(recall_score(y_pos, y_pos_pred, average = 'macro')*100, 2))
  print('Precision:', round(precision_score(y_pos, y_pos_pred, average = 'macro')*100, 2))
  print('F1:', round(f1_score(y_pos, y_pos_pred, average = 'macro')*100, 2))
  print('\n')

  print('NEUTRAL')
  print('\n')
  print('Recall:', round(recall_score(y_neu, y_neu_pred, average = 'macro')*100, 2))
  print('Precision:', round(precision_score(y_neu, y_neu_pred, average = 'macro')*100, 2))
  print('F1:', round(f1_score(y_neu, y_neu_pred, average = 'macro')*100, 2))
  print('\n')

  print('NEGATIVE')
  print('\n')
  print('Recall:', round(recall_score(y_neg, y_neg_pred, average = 'macro')*100, 2))
  print('Precision:', round(precision_score(y_neg, y_neg_pred, average = 'macro')*100, 2))
  print('F1:', round(f1_score(y_neg, y_neg_pred, average = 'macro')*100, 2))
  print('\n')

  print('ACCURACY:', round(((y == y_pred)*1).mean()*100, 2))
  print('\n')

# 5. Load datasets

In [19]:
df_train = pd.read_csv(f'{path_general}/datasets/df_train.csv')

In [20]:
df_val = pd.read_csv(f'{path_general}/datasets/df_test.csv')

In [21]:
df_test = pd.read_csv(f'{path_general}/datasets/df_test.csv')

In [22]:
df_train.shape

(2907, 2)

In [23]:
df_val.shape

(970, 2)

In [24]:
df_test.shape

(970, 2)

In [25]:
X_train = pd.DataFrame(df_train.apply(lambda x: generate_test_prompt(x['text']), axis = 1)).rename(columns = {0: 'text'})
y_train = df_train['sentiment']

In [26]:
X_val = pd.DataFrame(df_val.apply(lambda x: generate_test_prompt(x['text']), axis = 1)).rename(columns = {0: 'text'})
y_val = df_val['sentiment']

In [27]:
X_test = pd.DataFrame(df_test.apply(lambda x: generate_test_prompt(x['text']), axis = 1)).rename(columns = {0: 'text'})
y_test = df_test['sentiment']

# 6. Load model

In [28]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [29]:
path_model = f'{path_general}/model'

In [30]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = path_model,
    device_map = 'cuda',
    offload_folder = path_model,
    token = HUGGING_FACE_TOKEN
    )

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [31]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          trust_remote_code = True,
                                          padding_side = 'left',
                                          add_bos_token = True,
                                          add_eos_token = True,
                                          token = HUGGING_FACE_TOKEN
                                          )

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [33]:
tokenizer.pad_token = tokenizer.eos_token

# 5. Inference

In [34]:
pipe = pipeline(task = "text-generation", model = model, tokenizer = tokenizer, max_new_tokens = 1, temperature = 0.0)

In [35]:
y_train_pred = X_train['text'].map(lambda x: predict_sentiment(x, pipe))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [36]:
y_val_pred = X_val['text'].map(lambda x: predict_sentiment(x, pipe))

In [37]:
y_test_pred = X_test['text'].map(lambda x: predict_sentiment(x, pipe))

# 6. Performance

In [38]:
eval_performance(y_train, y_train_pred)

POSITIVE


Recall: 90.65
Precision: 91.49
F1: 91.06


NEUTRAL


Recall: 90.72
Precision: 90.84
F1: 90.78


NEGATIVE


Recall: 96.21
Precision: 94.88
F1: 95.53


ACCURACY: 90.95




In [39]:
eval_performance(y_val, y_val_pred)

POSITIVE


Recall: 88.31
Precision: 90.17
F1: 89.16


NEUTRAL


Recall: 88.29
Precision: 88.49
F1: 88.38


NEGATIVE


Recall: 96.16
Precision: 91.71
F1: 93.77


ACCURACY: 88.56




In [40]:
eval_performance(y_test, y_test_pred)

POSITIVE


Recall: 87.74
Precision: 89.48
F1: 88.54


NEUTRAL


Recall: 87.77
Precision: 87.95
F1: 87.86


NEGATIVE


Recall: 96.16
Precision: 91.71
F1: 93.77


ACCURACY: 88.04




### End of execution

In [41]:
end = time.time()

delta = (end - start)

hours = int(delta/3_600)
mins = int((delta - hours*3_600)/60)
secs = int(delta - hours*3_600 - mins*60)

print(f'Hours: {hours}, Minutes: {mins}, Seconds: {secs}')

Hours: 0, Minutes: 13, Seconds: 23
